In [4]:
import numpy as np

f = "/media/george-vengrovski/flash-drive/labels_SHAM_USA5271.npz"

f = np.load(f, allow_pickle=True)

print(f.files)
# hdbscan_labels, dataloader_indices, sample_ids, data_dir_mapping

print("hdbscan_labels shape:", f['hdbscan_labels'].shape)
print("dataloader_indices shape:", f['dataloader_indices'].shape)
print("sample_ids shape:", f['sample_ids'].shape)
print("data_dir_mapping shape:", f['data_dir_mapping'].shape)

['embedding_outputs', 'hdbscan_labels', 'ground_truth_labels', 'specs', 'hdbscan_colors', 'ground_truth_colors', 'dataloader_indices', 'dataloader_colors', 'sample_ids', 'data_dir_mapping']
hdbscan_labels shape: (7072,)
ground_truth_colors shape: (30,)
dataloader_indices shape: (7072,)
sample_ids shape: (2072,)
data_dir_mapping shape: (2072,)
